In [ ]:
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 1.2 MB/s 
     |████████████████████████████████| 453 kB 46.9 MB/s 


In [ ]:
import pandas as pd
import numpy as np

from collections import Counter
from konlpy.tag import Okt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.models import load_model

from tensorflow.keras.layers import Embedding, Dense, LSTM, Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [1]:
from google.colab import files
file_uploaded = files.upload()

Saving unsmile_train_v1.0.tsv to unsmile_train_v1.0.tsv
Saving unsmile_valid_v1.0.tsv to unsmile_valid_v1.0.tsv


In [3]:
import pandas as pd
df_train = pd.read_csv("unsmile_train_v1.0.tsv",delimiter='\t')
df_valid = pd.read_csv("unsmile_valid_v1.0.tsv",delimiter='\t')

In [4]:
df_train = df_train.drop(['여성/가족', '남성', '성소수자', '인종/국적', '연령', '지역', '종교', '기타 혐오','악플/욕설','개인지칭'], axis=1)
df_valid = df_valid.drop(['여성/가족', '남성', '성소수자', '인종/국적', '연령', '지역', '종교', '기타 혐오','악플/욕설','개인지칭'], axis=1)

In [5]:
df_train['target'] = df_train['clean'].map({1:0, 0:1})
df_valid['target'] = df_valid['clean'].map({1:0, 0:1})

In [6]:
print(df_train['target'].value_counts())
print(df_valid['target'].value_counts())

1    11266
0     3739
Name: target, dtype: int64
1    2802
0     935
Name: target, dtype: int64


In [ ]:
okt = Okt()

In [ ]:
print('OKT 형태소 분석 :',okt.morphs("열심히 코딩한 당신, 연휴에는 여행을 가봐요"))
print('OKT 품사 태깅 :',okt.pos("열심히 코딩한 당신, 연휴에는 여행을 가봐요"))
print('OKT 명사 추출 :',okt.nouns("열심히 코딩한 당신, 연휴에는 여행을 가봐요"))

OKT 형태소 분석 : ['열심히', '코딩', '한', '당신', ',', '연휴', '에는', '여행', '을', '가봐요']
OKT 품사 태깅 : [('열심히', 'Adverb'), ('코딩', 'Noun'), ('한', 'Josa'), ('당신', 'Noun'), (',', 'Punctuation'), ('연휴', 'Noun'), ('에는', 'Josa'), ('여행', 'Noun'), ('을', 'Josa'), ('가봐요', 'Verb')]
OKT 명사 추출 : ['코딩', '당신', '연휴', '여행']


In [ ]:
def make_corpus(sentence):
    okt = Okt()
    raw_pos_tagged = okt.pos(sentence, norm=True, stem=True)
    word_cleaned = []

    for word in raw_pos_tagged:
        if word[1] in ['Noun', 'Adjective', 'Verb']:
            word_cleaned.append(word[0])
    return ' '.join(word_cleaned)

In [ ]:
df_train['tokenized'] = df_train['문장'].map(make_corpus)
df_valid['tokenized'] = df_valid['문장'].map(make_corpus)

In [ ]:
tfidf_vect = TfidfVectorizer(max_features=100)
tfidf_vect.fit(df_train.tokenized)
df_train_tfidf = tfidf_vect.transform(df_train.tokenized)
df_valid_tfidf = tfidf_vect.transform(df_valid.tokenized)

In [ ]:
df_train_tfidf = df_train_tfidf.toarray()
df_valid_tfidf = df_valid_tfidf.toarray()

In [ ]:
X_train = df_train_tfidf
X_test = df_valid_tfidf
y_train = df_train['target']
y_test = df_valid['target']

In [ ]:
print("X_train shape: ", X_train.shape)
print("X_test shape: ", X_test.shape)
print("y_train shape: ", y_train.shape)
print("y_test shape: ", y_test.shape)

X_train shape:  (15005, 100)
X_test shape:  (3737, 100)
y_train shape:  (15005,)
y_test shape:  (3737,)


In [ ]:
embedding_dim = 100
hidden_units = 128

model = Sequential()
model.add(Embedding(6, embedding_dim))
# model.add(Bidirectional(LSTM(hidden_units))) # Bidirectional LSTM을 사용
model.add(LSTM(hidden_units))
model.add(Dense(1, activation='sigmoid'))

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_model_old.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train, y_train, epochs=5, callbacks=[es, mc], batch_size=256, validation_split=0.2)

Epoch 1/5
47/47 [==============================] - ETA: 0s - loss: 0.5757 - acc: 0.7421
Epoch 1: val_acc improved from -inf to 0.74375, saving model to best_model_old.h5
47/47 [==============================] - 33s 653ms/step - loss: 0.5757 - acc: 0.7421 - val_loss: 0.5700 - val_acc: 0.7438
Epoch 2/5
47/47 [==============================] - ETA: 0s - loss: 0.5614 - acc: 0.7526
Epoch 2: val_acc did not improve from 0.74375
47/47 [==============================] - 30s 649ms/step - loss: 0.5614 - acc: 0.7526 - val_loss: 0.5702 - val_acc: 0.7438
Epoch 3/5
47/47 [==============================] - ETA: 0s - loss: 0.5605 - acc: 0.7526
Epoch 3: val_acc did not improve from 0.74375
47/47 [==============================] - 30s 630ms/step - loss: 0.5605 - acc: 0.7526 - val_loss: 0.5705 - val_acc: 0.7438
Epoch 4/5
47/47 [==============================] - ETA: 0s - loss: 0.5595 - acc: 0.7526
Epoch 4: val_acc did not improve from 0.74375
47/47 [==============================] - 29s 628ms/step - loss

In [ ]:
loaded_model = load_model('best_model_old.h5')

print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

117/117 [==============================] - 10s 74ms/step - loss: 0.5628 - acc: 0.7498

 테스트 정확도: 0.7498


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 100)         600       
                                                                 
 lstm (LSTM)                 (None, 128)               117248    
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 117,977
Trainable params: 117,977
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# preds = model.predict(X_test)

preds_2 = loaded_model.predict(X_test)

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

# clf = LogisticRegression(solver="liblinear", random_state=0).fit(X_test, y_test)

# roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1]) 
AUC = roc_auc_score(y_test, preds_2)     

# y_pred = model.predict(test_labels)
precision = precision_score(y_test, np.round(abs(preds_2))) 
f1 = f1_score(y_test, np.round(abs(preds_2))) 

print("AUC : ", AUC)
print("Precision Score : ", precision)
print("f1 Score : ", f1)

AUC :  0.547882146824079
Precision Score :  0.7497993042547498
f1 Score :  0.8570117755008412


In [ ]:
embedding_dim = 100
hidden_units = 128

model = Sequential()
model.add(Embedding(6, embedding_dim))
model.add(Bidirectional(LSTM(hidden_units))) # Bidirectional LSTM을 사용
# model.add(LSTM(hidden_units))
model.add(Dense(1, activation='sigmoid'))

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_model_old.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train, y_train, epochs=5, callbacks=[es, mc], batch_size=256, validation_split=0.2)

Epoch 1/5
47/47 [==============================] - ETA: 0s - loss: 0.5696 - acc: 0.7416
Epoch 1: val_acc improved from -inf to 0.74375, saving model to best_model_old.h5
47/47 [==============================] - 67s 1s/step - loss: 0.5696 - acc: 0.7416 - val_loss: 0.5728 - val_acc: 0.7438
Epoch 2/5
47/47 [==============================] - ETA: 0s - loss: 0.5604 - acc: 0.7526
Epoch 2: val_acc did not improve from 0.74375
47/47 [==============================] - 56s 1s/step - loss: 0.5604 - acc: 0.7526 - val_loss: 0.5706 - val_acc: 0.7438
Epoch 3/5
47/47 [==============================] - ETA: 0s - loss: 0.5608 - acc: 0.7526
Epoch 3: val_acc did not improve from 0.74375
47/47 [==============================] - 56s 1s/step - loss: 0.5608 - acc: 0.7526 - val_loss: 0.5675 - val_acc: 0.7438
Epoch 4/5
47/47 [==============================] - ETA: 0s - loss: 0.5585 - acc: 0.7526
Epoch 4: val_acc did not improve from 0.74375
47/47 [==============================] - 56s 1s/step - loss: 0.5585 - a

In [ ]:
loaded_model = load_model('best_model_old.h5')

print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

117/117 [==============================] - 7s 56ms/step - loss: 0.5675 - acc: 0.7498

 테스트 정확도: 0.7498


In [ ]:
preds = loaded_model.predict(X_test)

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

# clf = LogisticRegression(solver="liblinear", random_state=0).fit(X_test, y_test)

# roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1]) 
AUC = roc_auc_score(y_test, preds)     

# y_pred = model.predict(test_labels)
precision = precision_score(y_test, np.round(abs(preds))) 
f1 = f1_score(y_test, np.round(abs(preds))) 

print("AUC : ", AUC)
print("Precision Score : ", precision)
print("f1 Score : ", f1)

AUC :  0.5490470901227925
Precision Score :  0.7497993042547498
f1 Score :  0.8570117755008412


In [ ]:
# 모델 구조 그림
pip install graphviz
pip install pydot

In [ ]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

%matplotlib inline

SVG(model_to_dot(model, show_shapes=True).create(prog='dot', format='svg'))